<a href="https://colab.research.google.com/github/MariaPosadaBotero/Entregas-del-proyecto-de-AI-para-ingenieros---Udea-2025-2/blob/main/99-modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MODELO COMPLETO CON LIGHTGBM
###INTEGRANTES:

*   Maria Isabel Posada Botero. CC: 1040355409.
* Diana Milena Oquendo Restrepo. CC: 1193123469.



##PREPROCESADO


In [ ]:
# Crear carpeta oculta para las credenciales
!mkdir -p ~/.kaggle

# Copiar el archivo kaggle.json que subiste
!cp kaggle.json ~/.kaggle/

# Ajustar permisos de seguridad
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# Descargo la libreria de kaggle
!pip install kaggle

In [ ]:
#Se descarga los datos de la competencia
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip, udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip.zip or udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip.ZIP.


In [ ]:
#Importar las librerias necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

#Cargar el dataset de entrenamiento
df = pd.read_csv("train.csv")
print("Tamaño del dataset:", df.shape)
df.head()


Tamaño del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
 #Limpieza básica

# Eliminar duplicados
df.drop_duplicates(inplace=True)

# Eliminar columnas irrelevantes (por ejemplo, ID)
if 'ID' in df.columns:
    df.drop(columns=['ID'], inplace=True)

# Corregir espacios en blanco y tipos de datos
for c in df.select_dtypes(include='object'):
    df[c] = df[c].astype(str).str.strip()

# Verificar si quedaron valores faltantes
print("Valores faltantes por columna:")
print(df.isnull().sum())

# Mostrar las primeras filas del dataset limpio
print("\nVista previa del dataset después de la limpieza básica:")
display(df.head())

# Mostrar el tamaño del dataset limpio
print("\nTamaño del dataset después de la limpieza:", df.shape)

Valores faltantes por columna:
PERIODO_ACADEMICO              0
E_PRGM_ACADEMICO               0
E_PRGM_DEPARTAMENTO            0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
F_TIENEAUTOMOVIL               0
E_PRIVADO_LIBERTAD             0
E_PAGOMATRICULAPROPIO          0
F_TIENECOMPUTADOR              0
F_TIENEINTERNET.1              0
F_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL             0
INDICADOR_1                    0
INDICADOR_2                    0
INDICADOR_3                    0
INDICADOR_4                    0
dtype: int64

Vista previa del dataset después de la limpieza básica:


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294



Tamaño del dataset después de la limpieza: (692500, 20)


In [ ]:
#Imputación de valores faltantes

# Separar numéricas y categóricas
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude=np.number).columns

# Imputar numéricas con la media
imputer_num = SimpleImputer(strategy='mean')
df[num_cols] = imputer_num.fit_transform(df[num_cols])

# Imputar categóricas con el valor más frecuente
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

# Verificación
df.isnull().sum()


,0
PERIODO_ACADEMICO,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_VALORMATRICULAUNIVERSIDAD,0
E_HORASSEMANATRABAJA,0
F_ESTRATOVIVIENDA,0
F_TIENEINTERNET,0
F_EDUCACIONPADRE,0
F_TIENELAVADORA,0
F_TIENEAUTOMOVIL,0


In [ ]:
# Detectar columnas categóricas
cat_cols = df.select_dtypes(include=['object', 'category']).columns
print("Columnas categóricas detectadas:")
print(cat_cols)

# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=False)

# Mostrar información después del One-Hot Encoding
print("\nColumnas después del One-Hot Encoding:")
print(df_encoded.columns)

print("\nTamaño del dataset después del One-Hot Encoding:", df_encoded.shape)

print("\nVista previa de los primeros registros:")
display(df_encoded.head())


Columnas categóricas detectadas:
Index(['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')

Columnas después del One-Hot Encoding:
Index(['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3',
       'INDICADOR_4',
       'E_PRGM_ACADEMICO_3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES',
       'E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE',
       'E_PRGM_ACADEMICO_ACUICULTURA', 'E_PRGM_ACADEMICO_ADMINISTRACION',
       'E_PRGM_ACADEMICO_ADMINISTRACION  FINANCIERA',
       ...
       'F_EDUCACIONMADRE_Primaria incompleta',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',

,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,E_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,F_EDUCACIONMADRE_nan,RENDIMIENTO_GLOBAL_alto,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,20212.0,0.322,0.208,0.310,0.267,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,20212.0,0.311,0.215,0.292,0.264,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
2,20203.0,0.297,0.214,0.305,0.264,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
3,20195.0,0.485,0.172,0.252,0.190,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
4,20212.0,0.316,0.232,0.285,0.294,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
# Normalización de valores numéricos

# Detectar columnas numéricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
print("Columnas numéricas detectadas:")
print(num_cols.tolist())

# Aplicar normalización Min-Max
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Mostrar una vista previa
print("\nVista previa del DataFrame normalizado:")
display(df.head())

Columnas numéricas detectadas:
['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

Vista previa del DataFrame normalizado:


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,0.966667,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.490107,0.427105,0.968750,0.804217
1,0.966667,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.473364,0.441478,0.912500,0.795181
2,0.666667,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.452055,0.439425,0.953125,0.795181
3,0.400000,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.738204,0.353183,0.787500,0.572289
4,0.966667,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.480974,0.476386,0.890625,0.885542


In [ ]:
# Detección y tratamiento de outliers
outlier_info = {}

for col in num_cols:
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    low = q1 - 1.5 * iqr
    high = q3 + 1.5 * iqr

    # Contar cuántos outliers había antes
    outliers = ((df[col] < low) | (df[col] > high)).sum()
    outlier_info[col] = outliers

    # Reemplazar valores fuera del rango con los límites
    df[col] = np.clip(df[col], low, high)

# Mostrar columnas donde sí había outliers
print("Número de valores atípicos detectados por columna:\n")
found_outliers = False
for col, count in outlier_info.items():
    if count > 0:
        print(f"{col}: {count} outliers corregidos")
        found_outliers = True

if not found_outliers:
    print("No se detectaron valores atípicos en las columnas numéricas.")

# Vista previa del dataset actualizado
print("\nVista previa del dataset después del tratamiento de outliers:\n")
display(df.head())

Número de valores atípicos detectados por columna:

INDICADOR_1: 68120 outliers corregidos
INDICADOR_2: 34162 outliers corregidos
INDICADOR_3: 36488 outliers corregidos
INDICADOR_4: 69596 outliers corregidos

Vista previa del dataset después del tratamiento de outliers:



,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,0.966667,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto,0.490107,0.427105,0.968750,0.804217
1,0.966667,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.473364,0.441478,0.912500,0.795181
2,0.666667,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.452055,0.439425,0.953125,0.795181
3,0.400000,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.731355,0.353183,0.787500,0.572289
4,0.966667,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo,0.480974,0.476386,0.890625,0.885542


In [ ]:
# Crear variable objetivo (target)

# Definir 'RENDIMIENTO_GLOBAL' como variable objetivo
if 'RENDIMIENTO_GLOBAL' in df.columns:
    # Convertir a valores numéricos (si es categórica)
    df['target'] = df['RENDIMIENTO_GLOBAL'].map({
        'bajo': 0,
        'medio-bajo': 1,
        'medio-alto': 2,
        'alto': 3
    })

    # Verificar si hubo valores no reconocidos
    print("Valores únicos en 'RENDIMIENTO_GLOBAL':", df['RENDIMIENTO_GLOBAL'].unique())
    print("\nDistribución de la variable objetivo:")
    print(df['target'].value_counts(dropna=False))
else:
    print("No se encontró la columna 'RENDIMIENTO_GLOBAL' en el dataset.")


Valores únicos en 'RENDIMIENTO_GLOBAL': ['medio-alto' 'bajo' 'alto' 'medio-bajo']

Distribución de la variable objetivo:
target
3    175619
0    172987
1    172275
2    171619
Name: count, dtype: int64


In [ ]:
#Preparar datos para modelado

# Verificar si existe una columna 'target'
if 'target' in df.columns:
    target = 'target'
    df = df[[c for c in df.columns if c != target] + [target]]
    print(f" Variable objetivo detectada: '{target}'")
    print(f"Distribución de la variable objetivo:\n{df[target].value_counts()}")
else:
    print(" No se encontró la columna 'target' en el dataset.")

print(f"\nDimensiones del dataset preparado: {df.shape}")
display(df.head())

 Variable objetivo detectada: 'target'
Distribución de la variable objetivo:
target
3    175619
0    172987
1    172275
2    171619
Name: count, dtype: int64

Dimensiones del dataset preparado: (692500, 21)


,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,...,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,target
0,0.966667,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,...,No,Si,Si,Postgrado,medio-alto,0.490107,0.427105,0.968750,0.804217,2
1,0.966667,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,...,No,Si,No,Técnica o tecnológica incompleta,bajo,0.473364,0.441478,0.912500,0.795181,0
2,0.666667,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,...,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.452055,0.439425,0.953125,0.795181,0
3,0.400000,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,...,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.731355,0.353183,0.787500,0.572289,3
4,0.966667,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,...,No,Si,Si,Primaria completa,medio-bajo,0.480974,0.476386,0.890625,0.885542,1


In [ ]:
#Guardar el dataset preprocesado

import os

# Verificar las dimensiones del dataset final antes de guardar
print("Dimensiones finales del dataset preprocesado:", df.shape)

# Guardar el DataFrame preprocesado en un archivo CSV
df.to_csv("train_preprocesado.csv", index=False, encoding='utf-8-sig')

# Confirmación visual
print("Archivo preprocesado guardado correctamente como 'train_preprocesado.csv'")

# Verificar que el archivo se haya guardado correctamente
if "train_preprocesado.csv" in os.listdir():
    print("Archivo encontrado en el directorio actual.")
else:
    print("No se encontró el archivo. Verifica la ruta o permisos.")

Dimensiones finales del dataset preprocesado: (692500, 21)
Archivo preprocesado guardado correctamente como 'train_preprocesado.csv'
Archivo encontrado en el directorio actual.


###PREPROCESADO DEL TEST


In [ ]:
import pandas as pd
import joblib

# Cargar el dataset preprocesado del train
train_preprocesado = pd.read_csv("train_preprocesado.csv")

# Extraer y guardar los nombres de las columnas
train_cols = train_preprocesado.columns.tolist()

# Guardar para reutilizar fácilmente
joblib.dump(train_cols, "train_columns.pkl")

print(f"{len(train_cols)} columnas guardadas en 'train_columns.pkl'")
print(train_cols)


21 columnas guardadas en 'train_columns.pkl'
['PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4', 'target']


In [ ]:
#  Importar librerías
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Cargar datos de test
test = pd.read_csv("test.csv")
print("Tamaño original del test:", test.shape)

#  Limpieza básica
test.drop_duplicates(inplace=True)
if 'ID' in test.columns:
    test.drop(columns=['ID'], inplace=True)

for c in test.select_dtypes(include='object'):
    test[c] = test[c].astype(str).str.strip()

print("\nValores faltantes por columna antes de imputar:")
print(test.isnull().sum())

# Imputación de valores faltantes
num_cols = test.select_dtypes(include=np.number).columns
cat_cols = test.select_dtypes(exclude=np.number).columns

# Reutiliza los mismos métodos que en el train
imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

test[num_cols] = imputer_num.fit_transform(test[num_cols])
test[cat_cols] = imputer_cat.fit_transform(test[cat_cols])

#  One-Hot Encoding
test_encoded = pd.get_dummies(test, columns=cat_cols, drop_first=False)

#Asegurar que tenga las mismas columnas que el train
import joblib
train_cols = joblib.load("train_columns.pkl")


#  Escalado numérico
scaler = MinMaxScaler()
num_cols_scaled = test_encoded.select_dtypes(include=['int64', 'float64']).columns
test_encoded[num_cols_scaled] = scaler.fit_transform(test_encoded[num_cols_scaled])

# Tratamiento de outliers
for col in num_cols_scaled:
    q1, q3 = test_encoded[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    low, high = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    test_encoded[col] = np.clip(test_encoded[col], low, high)

print("\nTamaño final del test preprocesado:", test_encoded.shape)
display(test_encoded.head())

#  Guardar el test preprocesado
test_encoded.to_csv("test_preprocesado.csv", index=False)
print("\n Archivo 'test_preprocesado.csv' guardado con éxito.")
print("Tamaño final:", test_encoded.shape)

Tamaño original del test: (296786, 20)

Valores faltantes por columna antes de imputar:
PERIODO_ACADEMICO              0
E_PRGM_ACADEMICO               0
E_PRGM_DEPARTAMENTO            0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
F_TIENEAUTOMOVIL               0
E_PRIVADO_LIBERTAD             0
E_PAGOMATRICULAPROPIO          0
F_TIENECOMPUTADOR              0
F_TIENEINTERNET.1              0
F_EDUCACIONMADRE               0
INDICADOR_1                    0
INDICADOR_2                    0
INDICADOR_3                    0
INDICADOR_4                    0
dtype: int64

Tamaño final del test preprocesado: (296786, 1024)


,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_3° CICLO PROFESIONAL NEGOCIOS INTERNACIONALES,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,E_PRGM_ACADEMICO_ADMINISTRACION DE EMPRESAS.,...,F_EDUCACIONMADRE_No Aplica,F_EDUCACIONMADRE_No sabe,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,F_EDUCACIONMADRE_nan
0,0.000000,0.494721,0.452479,0.984472,0.746224,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,0.666667,0.342383,0.584711,0.919255,0.978852,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,0.966667,0.429864,0.471074,0.913043,0.746224,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,0.400000,0.241327,0.842975,0.673913,0.888218,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,0.966667,0.315234,0.584711,0.950311,0.864048,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False



 Archivo 'test_preprocesado.csv' guardado con éxito.
Tamaño final: (296786, 1024)


##MODELO LIGHTGBM

In [ ]:
# 1. IMPORTAR LIBRERIAS

import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# 2. CARGAR DATOS

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)


Train shape: (692500, 21)
Test shape: (296786, 20)


In [ ]:
# 3. DEFINIR LA VARIABLE OBJETIVO
target_col = "RENDIMIENTO_GLOBAL"

print("Valores únicos del target:", train[target_col].unique())

Valores únicos del target: ['medio-alto' 'bajo' 'alto' 'medio-bajo']


In [ ]:
# 4. QUITAR FILAS CON TARGET VACÍO

train = train.dropna(subset=[target_col])


In [ ]:
# 5. SEPARAR X y Y

X = train.drop(columns=[target_col])
y = train[target_col]

In [ ]:
# 6. IDENTIFICAR COLUMNAS CATEGÓRICAS
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
print("Columnas categóricas:", cat_cols)

# Convertir a categoría
for col in cat_cols:
    X[col] = X[col].astype("category")
    test[col] = test[col].astype("category")


Columnas categóricas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE']


In [ ]:
# 7. DIVIDIR EN TRAIN / VALIDATION
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# 8. CREAR MODELO LIGHTGBM

params = {
    "objective": "multiclass",
    "num_class": y.nunique(),
    "learning_rate": 0.08,
    "num_leaves": 40,
    "max_depth": -1,
    "n_estimators": 200,
    "verbose": -1
}

model = lgb.LGBMClassifier(**params)


In [ ]:
# 9. ENTRENAR
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="multi_logloss"
)

LGBMClassifier(learning_rate=0.08, n_estimators=200, num_class=4, num_leaves=40,
               objective='multiclass', verbose=-1)

In [ ]:

# 10. EVALUACIÓN: TRAIN Y VALIDACIÓN

# Predicciones en train
pred_train = model.predict(X_train)
acc_train = accuracy_score(y_train, pred_train)
print("Accuracy en TRAIN:", acc_train)

# Predicciones en validación
pred_val = model.predict(X_val)
acc_val = accuracy_score(y_val, pred_val)
print("Accuracy en VALIDACIÓN:", acc_val)

print("\n===== CLASSIFICATION REPORT VALIDACIÓN =====")
print(classification_report(y_val, pred_val))

Accuracy en TRAIN: 0.4901606498194946
Accuracy en VALIDACIÓN: 0.43937184115523464

===== CLASSIFICATION REPORT VALIDACIÓN =====
              precision    recall  f1-score   support

        alto       0.57      0.62      0.59     35124
        bajo       0.47      0.57      0.52     34597
  medio-alto       0.33      0.27      0.30     34324
  medio-bajo       0.33      0.28      0.31     34455

    accuracy                           0.44    138500
   macro avg       0.43      0.44      0.43    138500
weighted avg       0.43      0.44      0.43    138500



In [ ]:
# 11. PREDICCIÓN PARA KAGGLE

pred_test = model.predict(test)

submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": pred_test
})

submission.to_csv("submission_lightgbm.csv", index=False)
print("\nArchivo submission_lightgbm.csv generado")


Archivo submission_lightgbm.csv generado
